In [4]:
import torch
import gym
from torch import nn
import random
from sb3_contrib import TRPO

ImportError: cannot import name 'BaseAlgorithm' from partially initialized module 'stable_baselines3.common.base_class' (most likely due to a circular import) (C:\Users\Tobias\anaconda3\envs\loool\lib\site-packages\stable_baselines3\common\base_class.py)

In [38]:
class ClipReward(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.best_y = 6
        self.score = 0
        self.last_y = 6

    def reward(self, reward):
        ram = env.unwrapped.ale.getRAM()
        reward = 0
        if 140 <= ram[106] <= 141:
            reward += 10000
        reward += (ram[14] * 1.5) - 90
        if 90 <= ram[106] <= 100:
            reward -= 1000
        if ram[14] >= 172:
            self.best_y = 6
        return reward

class ObservationRAM(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)

    def observation(self, reward):
        ram = env.unwrapped.ale.getRAM()
        print(type(ram))
        ram = ram[[14, 103, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117]].astype(float)

        ram[0] /= 176
        ram[2] /= 142
        for i in range(3, 13):
            ram[i] /= 160
        return ram

In [39]:
env = ClipReward(ObservationRAM(gym.make("ALE/Freeway-v5", obs_type="ram", render_mode="human", difficulty=1, mode=3)))

env.reset()

<class 'numpy.ndarray'>


array([0.03409091, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])

In [4]:
def get_action_sample():
    x = random.randint(0, 101)
    if x < 90:
        return 1
    if x < 97:
        return 2
    return 0

In [12]:
env = ClipReward(gym.make("ALE/Freeway-v5", obs_type="ram", render_mode="human", difficulty=1, mode=7))
obs = env.reset()

for _ in range(1000):
    action = get_action_sample()
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
3
3
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
0
1000
0
0
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
3
3
3
0
3
3
3
3
3
3
3
3
3
3
1000
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
0
3
3
3
3
3
0
3
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
0
0
0
0
0
0
0
0
0
0
3
3
3
3
3
3
1000
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
3
3
3
3
3
0
0
3
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

In [ ]:
ITERATIONS = 50
STEPS_PER_ITERATION = 2048

env = ClipReward(gym.make("ALE/Freeway-v5", obs_type="ram", difficulty=1, mode=3))
obs = env.reset()

#model = TRPO("MlpPolicy", env, gamma=0.95, verbose=1, tensorboard_log="tensorlog")
model = TRPO.load("trpo_1.zip", env=env)
model.learn(total_timesteps=ITERATIONS*STEPS_PER_ITERATION, log_interval=10)
model.save("trpo_1.zip")

model = TRPO.load("trpo_1.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=3))
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

In [ ]:
%%time
ITERATIONS = 50
STEPS_PER_ITERATION = 2048

env = ClipReward(gym.make("ALE/Freeway-v5", obs_type="ram", difficulty=1, mode=3))
obs = env.reset()

model = TRPO("MlpPolicy", env, gamma=0.95, verbose=1)
model.learn(total_timesteps=ITERATIONS*STEPS_PER_ITERATION, log_interval=5)

In [24]:
model = TRPO.load("trpo_1.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=3))
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

0.0
0.0
3.666666666666666
-4.5
-10.666666666666668
-10.666666666666668
0.0
-10.666666666666668
0.0
0.0
6.857142857142857
12.444444444444445
17.454545454545453
22.153846153846153
26.666666666666668
31.058823529411764
35.368421052631575
39.61904761904762
43.82608695652174
-1010.6666666666666
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
0.0
6.857142857142857
12.444444444444445
17.454545454545453
22.153846153846153
26.666666666666668
31.058823529411764
35.368421052631575
39.61904761904762
43.82608695652174
48.0
43.82608695652174
48.0
52.148148148148145
48.0
52.148148148148145
48.0
52.148148148148145
48.0
52.148148148148145
56.275862068965516
60.38709677419355
64.48484848484848
68.57142857142857
72.64864864864865
76.71794871794872
80.78048780487805
84.83720930232558
88.8888888888

KeyboardInterrupt: 

In [ ]:
model = TRPO.load("trpo_3.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=0))
obs = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

In [22]:
import gym
from gym.wrappers import Monitor
import time

env = Monitor(ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=3)), './videoo', force=True)
state = env.reset()

model = TRPO.load("trpo_1.zip", env=env)

done = False
for i in range(1000):
    action, _states = model.predict(state, deterministic=True)
    state, reward, done, info = env.step(action)
    time.sleep(0.00001)
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
